In [ ]:
import pandas as pd
import os
import numpy as np
import io
from PIL import Image
import random

In [ ]:
NAME_FILE = 'example.csv'

In [ ]:
if not os.path.exists('DATA'):
    os.mkdir('DATA')
with open(os.path.join('DATA', NAME_FILE), 'w') as f:
    f.writelines(['COL_NAME\n']+[''.join([str(random.randint(0, 9)) for x in range(10)])+'DDDD\n' for _ in range(100000)])

In [ ]:
# For this example, read CSV-file with one column
df = pd.read_csv(os.path.join('DATA', NAME_FILE))

# Сhange the name of the original column
NEW_NAME = 'KEY16'
df.columns = [NEW_NAME]

NAME_COLUMNS = [f'col_{x}' for x in range(10)]

In [ ]:
# Add columns in DataFrame
for i, x in enumerate(NAME_COLUMNS): 
    df[x] = df[['KEY16']].apply(lambda row: row.KEY16[i:i+1], axis = 1)

In [ ]:
# Select all unique values from DataFrame
uniq_values = np.unique(df[NAME_COLUMNS].values)

In [ ]:
# Count of unique values for each column
dict_sum_value = {col_name:[sum(df[col_name] == value) for value in uniq_values] for col_name in NAME_COLUMNS}

df2 = pd.DataFrame(dict_sum_value, index = uniq_values)
df2

In [ ]:
writer = pd.ExcelWriter('report.xlsx', engine='xlsxwriter')
df2.to_excel(writer)

workbook = writer.book
worksheet_original = writer.sheets['Sheet1']

for i, name in enumerate(NAME_COLUMNS):
    buf = io.BytesIO()
    
    ax = df2.reset_index().plot(x='index', y=name, kind='bar', rot=0)
    ax.xaxis.set_label_text('')
    ax.figure.savefig(buf, format='png')
    
    buf.seek(0)
    im = Image.open(buf)

    worksheet_original.insert_image(f'A{15 + i *20}', f'image_{name}.png', {'image_data':buf})
    # im.show()
    
writer.save()
workbook.close()
buf.close()